In [ ]:
!apt install -y espeak-ng
!pip install torch torchaudio uv
!git clone https://github.com/Zyphra/Zonos.git
%cd Zonos
!pip install -e .

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 espeak-ng amd64 1.50+dfsg-1

In [ ]:
!pip install --upgrade numpy


In [ ]:
import torch
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
from zonos.utils import DEFAULT_DEVICE as device

model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

In [ ]:
import re

text = '''0011_000283	They had been named Tom and Jerry.	Neutral
0011_000284	Chapter eighteen, the return journey.	Neutral
0011_000285	The straw, charcoal and the pea.	Neutral
0011_000286	One two three, where is your breakfast.	Neutral
0011_000287	A voice said: Come in.	Neutral'''

parsed_lines = []
for line in text.split("\n"):
    match = re.match(r"^\S+\t(.*?)\tNeutral$", line)
    if match:
        parsed_lines.append(match.group(1))

print(parsed_lines)


['They had been named Tom and Jerry.', 'Chapter eighteen, the return journey.', 'The straw, charcoal and the pea.', 'One two three, where is your breakfast.', 'A voice said: Come in.']


In [ ]:
import torchaudio
import torch

# Загружаем референсный голос
wav, sampling_rate = torchaudio.load('/content/0011_merged.wav')
speaker = model.make_speaker_embedding(wav, sampling_rate)

# Определяем параметры эмоций (сбалансированные значения)
emotions = {
    "happy": [1, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077],      # 😊 Радость
    "sad": [0.3077, 1, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077],      # 😢 Грусть
    "angry": [0.3, 0.02, 0.02, 0.02, 0.02, 1, 0.2, 0.3],      # 😡 Злость
    "surprised": [0.3077, 0.0256, 0.0256, 0.0256, 1, 0.0256, 0.2564, 0.3077],  # 😲 Удивление
    "neutral": [0.3077, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077]      # 😐 Нейтральный тон
}

for num, text in enumerate(parsed_lines):
  for emotion_name, emotion_vector in emotions.items():
      # Создаём параметры для модели
      cond_dict = make_cond_dict(
          text=text,
          speaker=speaker,
          emotion=emotion_vector
      )

      # Генерируем голос
      conditioning = model.prepare_conditioning(cond_dict)
      codes = model.generate(conditioning)
      wavs = model.autoencoder.decode(codes).cpu()

      torchaudio.save(f'/content/drive/MyDrive/dataset/0011/{emotion_name}/{num + 283}.wav', wavs[0], model.autoencoder.sampling_rate)


ResNet293_SimAM_ASP_base.pt:   0%|          | 0.00/397M [00:00<?, ?B/s]

ResNet293_SimAM_ASP_base_LDA-128.pt:   0%|          | 0.00/265k [00:00<?, ?B/s]

Generating:   0%|          | 0/2588 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
W0324 21:41:45.349000 2740 torch/_inductor/utils.py:1137] [3/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipp

In [ ]:
import re

text = '''0012_000288	She has felt shy of it.	Neutral
0012_000289	No, said piggy carelessly.	Neutral
0012_000290	There goes the shawl again!	Neutral
0012_000291	The end of february fourteenth.	Neutral
0012_000292	She has a high voice.	Neutral
0012_000293	He is old fashioned but he is the best of men.	Neutral
0012_000294	Now I understand.	Neutral
0012_000295	I know how to obey orders.	Neutral
0012_000296	You woke me up!	Neutral
0012_000297	How Tom and Jerry went to visit mister Sam.	Neutral
0012_000298	She come back to the valley.	Neutral
0012_000299	His hope will cheat him.	Neutral
0012_000300	Six was half way down the room.	Neutral
0012_000301	I'm sure your friends can wait!	Neutral
0012_000302	Five hundred and one million right?	Neutral
0012_000303	I am loath to see him go.	Neutral
0012_000304	How do you like you dirty bird!	Neutral
0012_000305	The fisherman and his wife see George every day.	Neutral
0012_000306	Please invite Tom if there is not requires.	Neutral
0012_000307	I owe them five hundred dollars.	Neutral
0012_000308	Perhaps you think that is a queer title for this chapter.	Neutral
0012_000309	He owed mister Lawson, I can't tell you how.	Neutral
0012_000310	Why it is just like the round egg which sounds thin.	Neutral
0012_000311	I am safe back again.	Neutral
0012_000312	Father has yellow eyes.	Neutral
0012_000313	Give me a fine fat goose.	Neutral
0012_000314	Bob goes to a new school.	Neutral
0012_000315	It's to say how do you do with Tom's answer.	Neutral
0012_000316	A thief in the night.	Neutral
0012_000317	At length they came to a sharp bend.	Neutral
0012_000318	The name of the song is called haddocks.	Neutral
0012_000319	How could I dig burrows with short toe nails?	Neutral
0012_000320	Goat Billy asked the old chinese guy.	Neutral
0012_000321	Called out the cloud.	Neutral
0012_000322	A special pot too.	Neutral
0012_000323	Why did I wake up!	Neutral
0012_000324	And she caught at the brooch.	Neutral
0012_000325	She has no place for hot peper .	Neutral
0012_000326	They are made of wood.	Neutral
0012_000327	It's part of my secret.	Neutral
0012_000328	It would be a hard choice.	Neutral
0012_000329	And his heart wagged with joy like a lamb's tail.	Neutral
0012_000330	She keeps saying I'm hers.	Neutral
0012_000331	She'll be feverish after so much thinking.	Neutral
0012_000332	where are you going?	Neutral
0012_000333	You gain for nothing girl!	Neutral
0012_000334	Another one, ten thousand dollars.	Neutral
0012_000335	But things haven't change yet.	Neutral
0012_000336	Over them swooped the eagles.	Neutral
0012_000337	But mom I'm not certain about.	Neutral
0012_000338	Paul's teeth ached because of lemon.	Neutral
0012_000339	And be with you,Tom!	Neutral
0012_000340	So Tom saw night as it were broad daylight.	Neutral
0012_000341	Enough,you a foolish chatter.	Neutral
0012_000342	Why should I purchase my own?	Neutral
0012_000343	And what does it mean?	Neutral
0012_000344	Lucy, a clever farmer.	Neutral
0012_000345	A whirring noise was heard.	Neutral
0012_000346	I am going to back home.	Neutral
0012_000347	What are you waiting for? man.	Neutral
0012_000348	Dear customer,welcome to our ship.	Neutral
0012_000349	What do you think of this question?	Neutral
0012_000350	Do you have anything in mind?	Neutral'''

parsed_lines = []
for line in text.split("\n"):
    match = re.match(r"^\S+\t(.*?)\tNeutral$", line)
    if match:
        parsed_lines.append(match.group(1))

print(parsed_lines)


['She has felt shy of it.', 'No, said piggy carelessly.', 'There goes the shawl again!', 'The end of february fourteenth.', 'She has a high voice.', 'He is old fashioned but he is the best of men.', 'Now I understand.', 'I know how to obey orders.', 'You woke me up!', 'How Tom and Jerry went to visit mister Sam.', 'She come back to the valley.', 'His hope will cheat him.', 'Six was half way down the room.', "I'm sure your friends can wait!", 'Five hundred and one million right?', 'I am loath to see him go.', 'How do you like you dirty bird!', 'The fisherman and his wife see George every day.', 'Please invite Tom if there is not requires.', 'I owe them five hundred dollars.', 'Perhaps you think that is a queer title for this chapter.', "He owed mister Lawson, I can't tell you how.", 'Why it is just like the round egg which sounds thin.', 'I am safe back again.', 'Father has yellow eyes.', 'Give me a fine fat goose.', 'Bob goes to a new school.', "It's to say how do you do with Tom's ans

In [ ]:
import torchaudio
import torch

# Загружаем референсный голос
wav, sampling_rate = torchaudio.load('/content/0011_merged.wav')
speaker = model.make_speaker_embedding(wav, sampling_rate)

# Определяем параметры эмоций (сбалансированные значения)
emotions = {
    "happy": [1, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077],      # 😊 Радость
    "sad": [0.3077, 1, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077],      # 😢 Грусть
    "angry": [0.3, 0.02, 0.02, 0.02, 0.02, 1, 0.2, 0.3],      # 😡 Злость
    "surprised": [0.3077, 0.0256, 0.0256, 0.0256, 1, 0.0256, 0.2564, 0.3077],  # 😲 Удивление
    "neutral": [0.3077, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077]      # 😐 Нейтральный тон
}

for num, text in enumerate(parsed_lines):
  for emotion_name, emotion_vector in emotions.items():
      # Создаём параметры для модели
      cond_dict = make_cond_dict(
          text=text,
          speaker=speaker,
          emotion=emotion_vector
      )

      # Генерируем голос
      conditioning = model.prepare_conditioning(cond_dict)
      codes = model.generate(conditioning)
      wavs = model.autoencoder.decode(codes).cpu()

      torchaudio.save(f'/content/drive/MyDrive/dataset/0011/{emotion_name}/{num + 288}.wav', wavs[0], model.autoencoder.sampling_rate)


Generating:   8%|▊         | 195/2588 [00:18<03:51, 10.34it/s]

In [ ]:
import torchaudio
import torch

# Загружаем референсный голос
wav, sampling_rate = torchaudio.load('/content/0011_merged.wav')
speaker = model.make_speaker_embedding(wav, sampling_rate)

# Определяем параметры эмоций (сбалансированные значения)
emotions = {
    "neutral": [0.3, 0.02, 0.02, 0.02, 0.02, 0.02, 0.2, 0.3]
}

# Текст для генерации
text = "Then we all say aha! "

# Генерация аудио для каждой эмоции
for emotion_name, emotion_vector in emotions.items():
    # Создаём параметры для модели
    cond_dict = make_cond_dict(
        text=text,
        speaker=speaker,
        emotion=emotion_vector
    )

    # Генерируем голос
    conditioning = model.prepare_conditioning(cond_dict)
    codes = model.generate(conditioning)
    wavs = model.autoencoder.decode(codes).cpu()

    # Сохраняем результат
    filename = f"sample2_{emotion_name}.wav"
    torchaudio.save(filename, wavs[0], model.autoencoder.sampling_rate)

    print(f"✅ Сгенерировано аудио: {filename}")


Generating:   5%|▌         | 137/2588 [00:13<03:56, 10.36it/s]


✅ Сгенерировано аудио: sample2_neutral.wav


In [ ]:
import torchaudio
import torch

# Загружаем референсный голос
wav, sampling_rate = torchaudio.load('/content/0011_merged.wav')
speaker = model.make_speaker_embedding(wav, sampling_rate)

# Определяем параметры эмоций (сбалансированные значения)
emotions = {
    "happy": [1, 0.02, 0.02, 0.02, 0.02, 0.02, 0.2, 0.3],      # 😊 Радость
    "sad": [0.3, 1, 0.02, 0.02, 0.02, 0.02, 0.2, 0.3],      # 😢 Грусть
    "angry": [0.3, 0.02, 0.02, 0.02, 0.02, 1, 0.2, 0.3],      # 😡 Злость
    "surprised": [0.3, 0.02, 0.02, 0.02, 1, 0.02, 0.2, 0.3],  # 😲 Удивление
    "neutral": [0.3077, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077]      # 😐 Нейтральный тон
}

# Текст для генерации
text = "Then we all say aha! "

# Генерация аудио для каждой эмоции
for emotion_name, emotion_vector in emotions.items():
    # Создаём параметры для модели
    cond_dict = make_cond_dict(
        text=text,
        speaker=speaker,
        emotion=emotion_vector
    )

    # Генерируем голос
    conditioning = model.prepare_conditioning(cond_dict)
    codes = model.generate(conditioning)
    wavs = model.autoencoder.decode(codes).cpu()

    # Сохраняем результат
    filename = f"sample_{emotion_name}.wav"
    torchaudio.save(filename, wavs[0], model.autoencoder.sampling_rate)

    print(f"✅ Сгенерировано аудио: {filename}")
